#  Load packages

In [7]:
# skip reinstalling packages we already have
using Pkg

pkgs = [
    "MLJ", "MLJBase", "MLJModels", "MLJEnsembles", "MLJLinearModels",
    "DecisionTree", "MLJDecisionTreeInterface", "NaiveBayes", 
    "MLJNaiveBayesInterface", "EvoTrees", "CategoricalArrays", "Random",
    "LIBSVM", "MLJLIBSVMInterface", "Plots", "MLJModelInterface",
    "CSV", "DataFrames", "UrlDownload", "XGBoost","MultivariateStats","PrettyTables"
]

# Filter out packages already installed
missing_pkgs = filter(pkg -> !(pkg in keys(Pkg.project().dependencies)), pkgs)

if !isempty(missing_pkgs)
    println("Installing missing packages: ", missing_pkgs)
    Pkg.add(missing_pkgs)
else
    println(" All required packages are already installed.")
end


 All required packages are already installed.


# Import packages

In [8]:
using CSV, DataFrames, Random
using Flux
using CategoricalArrays
using Plots
using Random
using Statistics
using DataFrames
using MLJ
using MLJBase
import Logging
using DataFrames
Logging.disable_logging(Logging.Info)

#Load your library of functions
include("utils.jl")
# Set a global random seed for reproducibility
Random.seed!(42)

# --- Accuracy function ---
#accuracy(pred::Vector{Int}, truth::Vector{Int}) = mean(pred .== truth)

TaskLocalRNG()

# Load Data

In [9]:


df = CSV.read("./data/updated_pollution_dataset.csv", DataFrame)

# Some log
println("First 5 rows of df:")
show(df[1:5, :], allcols=true)


# Convert column 10 to categorical (in-place!)
df[!, 10] = categorical(df[!, 10])

# Get the column name as string
col10_name = string(names(df)[10])
println("Column 10 name: ", col10_name)

# Extract labels (categories) as strings
label_names = levels(df[!, 10])
println("Labels: ", label_names)

# Extract the integer codes of the categories
targets = Float32.(levelcode.(df[!, 10]))
inputs  = Matrix{Float32}(df[:, 1:9])



First 5 rows of df:
5×10 DataFrame
 Row │ Temperature  Humidity  PM2.5    PM10     NO2      SO2      CO       Proximity_to_Industrial_Areas  Population_Density  Air Quality 
     │ Float64      Float64   Float64  Float64  Float64  Float64  Float64  Float64                        Int64               String15    
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │        29.8      59.1      5.2     17.9     18.9      9.2     1.72                            6.3                 319  Moderate
   2 │        28.3      75.6      2.3     12.2     30.8      9.7     1.64                            6.0                 611  Moderate
   3 │        23.1      74.7     26.7     33.8     24.4     12.6     1.63                            5.2                 619  Moderate
   4 │        27.1      39.1      6.1      6.3     13.5      5.3     1.15                           11.1                 551  Good
   5 │      

5000×9 Matrix{Float32}:
 29.8   59.1    5.2   17.9  18.9   9.2  1.72   6.3  319.0
 28.3   75.6    2.3   12.2  30.8   9.7  1.64   6.0  611.0
 23.1   74.7   26.7   33.8  24.4  12.6  1.63   5.2  619.0
 27.1   39.1    6.1    6.3  13.5   5.3  1.15  11.1  551.0
 26.5   70.7    6.9   16.0  21.9   5.6  1.01  12.7  303.0
 39.4   96.6   14.6   35.5  42.9  17.9  1.82   3.1  674.0
 41.7   82.5    1.7   15.8  31.1  12.7  1.8    4.6  735.0
 31.0   59.6    5.0   16.8  24.2  13.6  1.38   6.3  443.0
 29.4   93.8   10.3   22.7  45.1  11.8  2.03   5.4  486.0
 33.2   80.5   11.1   24.4  32.0  15.3  1.69   4.9  535.0
 26.3   65.7    1.3    5.5  18.3   5.9  0.85  13.0  529.0
 32.5   51.2    1.6   10.5  21.6  19.3  1.53   5.9  519.0
 20.0   53.3    3.7   12.9  26.1   6.6  1.09  10.2  538.0
  ⋮                                ⋮                
 26.5   43.0   18.8   18.4  17.6   2.1  1.08  10.6  426.0
 44.4  102.1   35.4   54.7  55.5  25.5  1.79   3.0  772.0
 46.8   93.8   11.8   25.4  33.8  28.7  3.27   3.7  5

# Prepare data for ANN

In [10]:

N = size(df, 1)
trainIdx, valIdx, testIdx = holdOut(N, 0.1, 0.2)

println("Train indices: ", length(trainIdx))
println("Validation indices: ", length(valIdx))
println("Test indices: ", length(testIdx))
println("df size: ", size(df))

trainingInputs  = inputs[trainIdx, :]
valInputs       = inputs[valIdx, :]
testInputs      = inputs[testIdx, :]

trainingTargets = targets[trainIdx]
valTargets      = targets[valIdx]
testTargets     = targets[testIdx]


println("\n\nFirst 5 targets:")
println(targets[1:5])

println("Training inputs (first 5 rows):")
for i in 1:5
    println(trainingInputs[i, :])
end


Train indices: 3500
Validation indices: 500
Test indices: 1000
df size: (5000, 10)


First 5 targets:
Float32[3.0, 3.0, 3.0, 1.0, 1.0]
Training inputs (first 5 rows):
Float32[22.9, 50.0, 19.2, 23.4, 18.8, 6.0, 0.95, 12.2, 225.0]
Float32[36.8, 73.6, 36.0, 51.2, 24.5, 11.2, 1.53, 6.0, 630.0]
Float32[30.1, 49.0, 7.7, 14.5, 20.2, 7.6, 0.98, 11.0, 465.0]
Float32[46.8, 93.8, 11.8, 25.4, 33.8, 28.7, 3.27, 3.7, 589.0]
Float32[26.8, 55.7, 0.3, 6.2, 17.3, 5.6, 0.94, 11.4, 425.0]


# Normalize

In [11]:

println("\n\n=========== Normalizing Inputs ===========")

# Compute normalization parameters from TRAINING set only
normParams = calculateMinMaxNormalizationParameters(trainingInputs)

# Normalize training set IN PLACE
normalizeMinMax!(trainingInputs, normParams)

# Normalize validation set in place
normalizeMinMax!(valInputs, normParams)

# Normalize test set in place
#valInputs_normalized = normalizeMinMax(valInputs, normParams)
normalizeMinMax!(testInputs, normParams)


println("\nTraining inputs after normalization (first 5 rows):")
for i in 1:5
    println(trainingInputs[i, :])
end

println("\nValidation inputs after normalization (first 5 rows):")
for i in 1:5
    println(valInputs[i, :])
end

println("\nTest inputs after normalization (first 5 rows):")
for i in 1:5
    println(testInputs[i, :])
end


# Convert to float32 for Flux compatibility
trainingInputs = Float32.(trainingInputs)
valInputs = Float32.(valInputs)
testInputs = Float32.(testInputs)

# Clip values to [0,1] after normalization
valInputs .= clamp.(valInputs, 0f0, 1f0)
testInputs .= clamp.(testInputs, 0f0, 1f0)


# Values should only fall within [0,1]
@assert(all(minimum(trainingInputs, dims=1) .== 0))
@assert(all(maximum(trainingInputs, dims=1) .== 1))
@assert(all(minimum(valInputs, dims=1) .>= 0))
@assert(all(maximum(valInputs, dims=1) .<= 1))
@assert(all(minimum(testInputs, dims=1) .>= 0))
@assert(all(maximum(testInputs, dims=1) .<= 1))


println("Train inputs range per feature: ", (minimum(trainingInputs, dims=1), maximum(trainingInputs, dims=1)))
println("Validation inputs range per feature: ", (minimum(valInputs, dims=1), maximum(valInputs, dims=1)))
println("Test inputs range per feature: ", (minimum(testInputs, dims=1), maximum(testInputs, dims=1)))




=========== Normalizing Inputs ===========

Training inputs after normalization (first 5 rows):
Float32[0.21396396, 0.15108696, 0.06508475, 0.073804244, 0.1826241, 0.20612246, 0.09771987, 0.41630903, 0.046875]
Float32[0.527027, 0.40760866, 0.1220339, 0.16186254, 0.28368795, 0.3122449, 0.28664494, 0.1502146, 0.57421875]
Float32[0.37612614, 0.1402174, 0.026101694, 0.045612924, 0.20744681, 0.2387755, 0.107491866, 0.3648069, 0.359375]
Float32[0.7522523, 0.6271739, 0.04, 0.080139376, 0.44858155, 0.66938776, 0.85342014, 0.05150215, 0.5208333]
Float32[0.30180177, 0.21304348, 0.0010169492, 0.019322142, 0.15602835, 0.19795918, 0.094462544, 0.38197425, 0.30729166]

Validation inputs after normalization (first 5 rows):
Float32[0.3400901, 0.23043478, 0.062372882, 0.11023125, 0.429078, 0.36734694, 0.61563516, 0.10729614, 0.38671875]
Float32[0.46171173, 0.29782608, 0.18677966, 0.24928732, 0.5904255, 0.46938777, 0.43648207, 0.0472103, 0.3450521]
Float32[0.45495492, 0.17173915, 0.18745762, 0.1982895

# Full Dimesnion

## ANN

In [12]:
using Flux
using Statistics
using Plots

# --- Safe accuracy function ---
accuracy_score(pred::Vector{Int}, truth::Vector{Int}) = mean(pred .== truth)
accuracy = accuracy_score  # alias so trainClassANN can call accuracy(...)
# Alias for Flux losses
const Losses = Flux

architectures = [
    [4, 4],
    [8, 8],
    [16, 16],
    [10, 4],
    [10, 6, 4],
    [10, 8, 4],
    [10, 8, 6, 4],
    [10, 12, 6, 4]
]

learningRate = 0.01
epochs = 200

# --- Store all results ---
results = Dict{String, Tuple{Vector{Float32}, Vector{Float32}, Vector{Float32}}}()

# Extract unique classes for one-hot encoding
class_labels = unique(trainingTargets)
labels = string.(class_labels)

for topology in architectures
    println("\nTraining architecture: ", topology)

    # One-hot encode the targets
    trainingTargetsOH = oneHotEncoding(trainingTargets, class_labels)
    valTargetsOH      = oneHotEncoding(valTargets, class_labels)
    testTargetsOH     = oneHotEncoding(testTargets, class_labels)

    # Train the network
    finalAnn, trainLoss, valLoss, testLoss = trainClassANN(
        topology,
        (trainingInputs, trainingTargetsOH),
        validationDataset = (valInputs, valTargetsOH),
        testDataset = (testInputs, testTargetsOH),
        maxEpochs = epochs,
        learningRate = learningRate,
        showText = false
    )

    results[string(topology)] = (trainLoss, valLoss, testLoss)

    # --- Compute confusion matrix on test set ---
    testOutputs = finalAnn(testInputs')
    testPredictions = classifyOutputs(testOutputs')  # boolean matrix
    cm_metrics = confusionMatrix(testPredictions, testTargetsOH)

    printANNResult(
        topology,
        Dict("learningRate" => learningRate, "epochs" => epochs),
        cm_metrics,
        label_names
    )

end


Training architecture: [4, 4]

 Model: [4, 4]   
 Hyperparameters: Dict{String, Real}("epochs" => 200, "learningRate" => 0.01)
 Accuracy                : 0.647
 Error Rate              : 0.353
 Sensitivity/Recall      : 0.647
 Specificity             : 0.8452
 PPV                     : 0.487
 NPV                     : 0.919
 F1 Score                : 0.545

Confusion Matrix:
[383 0 0 0; 32 264 0 0; 0 103 0 0; 0 218 0 0]

┌───────────┬──────┬───────────┬──────────┬──────┐
│           │ Good │ Hazardous │ Moderate │ Poor │
├───────────┼──────┼───────────┼──────────┼──────┤
│      Good │  383 │         0 │        0 │    0 │
│ Hazardous │   32 │       264 │        0 │    0 │
│  Moderate │    0 │       103 │        0 │    0 │
│      Poor │    0 │       218 │        0 │    0 │
└───────────┴──────┴───────────┴──────────┴──────┘

Training architecture: [8, 8]

 Model: [8, 8]   
 Hyperparameters: Dict{String, Real}("epochs" => 200, "learningRate" => 0.01)
 Accuracy                : 0.847
 Erro

# SVM/KNN/DT

## Prepare data for SVM/KNN/DT

In [13]:

trainIdx, testIdx = holdOut(N, 0.2)
println("Train indices: ", length(trainIdx))
println("Test indices: ", length(testIdx))
println("df size: ", size(df))

trainingInputs  = inputs[trainIdx, :]
testInputs      = inputs[testIdx, :]

trainingTargets = targets[trainIdx]
testTargets     = targets[testIdx]

println("\n\nFirst 5 targets:")
println(targets[1:5])

println("Training inputs (first 5 rows):")
for i in 1:5
    println(trainingInputs[i, :])
end



println("\n\n=========== Normalizing Inputs ===========")

# Compute normalization parameters from TRAINING set only
normParams = calculateMinMaxNormalizationParameters(trainingInputs)

# Normalize training set IN PLACE
normalizeMinMax!(trainingInputs, normParams)

# Normalize test set in place
#valInputs_normalized = normalizeMinMax(valInputs, normParams)
normalizeMinMax!(testInputs, normParams)


println("\nTraining inputs after normalization (first 5 rows):")
for i in 1:5
    println(trainingInputs[i, :])
end

println("\nTest inputs after normalization (first 5 rows):")
for i in 1:5
    println(testInputs[i, :])
end

# Convert to float32 for Flux compatibility
trainingInputs = Float32.(trainingInputs)
testInputs = Float32.(testInputs)

# Clip values to [0,1] after normalization
testInputs .= clamp.(testInputs, 0f0, 1f0)
testInputs .= clamp.(testInputs, 0f0, 1f0)

# Convert directly to DataFrames for MLJ
train_df = DataFrame(trainingInputs, :auto)
test_df  = DataFrame(testInputs, :auto)

# Values should only fall within [0,1]
@assert(all(minimum(trainingInputs, dims=1) .== 0))
@assert(all(maximum(trainingInputs, dims=1) .== 1))
@assert(all(minimum(testInputs, dims=1) .>= 0))
@assert(all(maximum(testInputs, dims=1) .<= 1))


println("Train inputs range per feature: ", (minimum(trainingInputs, dims=1), maximum(trainingInputs, dims=1)))
println("Test inputs range per feature: ", (minimum(testInputs, dims=1), maximum(testInputs, dims=1)))


Train indices: 4000
Test indices: 1000
df size: (5000, 10)


First 5 targets:
Float32[3.0, 3.0, 3.0, 1.0, 1.0]
Training inputs (first 5 rows):
Float32[27.0, 45.3, 1.8, 14.1, 34.1, 11.4, 1.64, 6.5, 615.0]
Float32[24.2, 67.6, 19.4, 35.1, 24.2, 9.6, 1.91, 6.2, 422.0]
Float32[28.9, 83.5, 26.1, 36.1, 31.3, 10.9, 1.71, 6.5, 523.0]
Float32[22.9, 57.4, 15.9, 22.8, 20.6, 7.5, 0.91, 10.4, 336.0]
Float32[37.1, 53.3, 37.3, 51.0, 25.9, 10.0, 0.98, 9.0, 473.0]


=========== Normalizing Inputs ===========

Training inputs after normalization (first 5 rows):
Float32[0.300885, 0.10097718, 0.006101695, 0.045253165, 0.4643478, 0.34442267, 0.31578946, 0.17167382, 0.55526656]
Float32[0.2389381, 0.3431053, 0.06576271, 0.11170886, 0.29217392, 0.30919763, 0.40460527, 0.15879828, 0.30429128]
Float32[0.34292036, 0.51574373, 0.08847458, 0.11487342, 0.41565216, 0.3346379, 0.33881578, 0.17167382, 0.43563068]
Float32[0.210177, 0.23235613, 0.053898305, 0.07278481, 0.22956523, 0.26810175, 0.075657904, 0.33905578, 0.1

# SVM

In [14]:
using MLJ, LIBSVM, CategoricalArrays, MLJBase, DataFrames

# --- Default hyperparameters ---
default_svm = Dict(
    "gamma"  => 1.0,
    "degree" => Int32(3),
    "coef0"  => 0.0
)

# --- Hyperparameter search space ---
svm_search_space = [
    Dict("kernel"=>"linear",  "C"=>0.1),
    Dict("kernel"=>"linear",  "C"=>1.0),
    Dict("kernel"=>"linear",  "C"=>10.0),
    Dict("kernel"=>"rbf",     "C"=>1.0, "gamma"=>2.0),
    Dict("kernel"=>"rbf",     "C"=>10.0,"gamma"=>0.5),
    Dict("kernel"=>"sigmoid", "C"=>1.0, "gamma"=>1.0),
    Dict("kernel"=>"poly",    "C"=>1.0, "degree"=>3, "gamma"=>1.0),
    Dict("kernel"=>"poly",    "C"=>5.0, "degree"=>4, "gamma"=>0.5),
]

# --- Convert targets ---
train_y_cat = categorical(vec(trainingTargets))
test_y_cat  = categorical(vec(testTargets))

# --- Container for results ---
svm_results = []

# --- Wrap PCA / raw data as DataFrames ---
# Assume train_df and test_df are already prepared DataFrames of features
# e.g., train_df = DataFrame(pca_train, :auto), test_df = DataFrame(pca_test, :auto)

for hp in svm_search_space
    println("\n=== SVM experiment: kernel=$(hp["kernel"]) C=$(hp["C"]) ===")

    # Map string kernels to LIBSVM.Kernel enums
    kernel_enum = hp["kernel"] == "linear"  ? LIBSVM.Kernel.Linear  :
                  hp["kernel"] == "rbf"     ? LIBSVM.Kernel.RadialBasis :
                  hp["kernel"] == "sigmoid" ? LIBSVM.Kernel.Sigmoid :
                  hp["kernel"] == "poly"    ? LIBSVM.Kernel.Polynomial :
                  error("Unsupported kernel: $(hp["kernel"])")

    cost   = Float64(hp["C"])
    gamma  = Float64(get(hp, "gamma", 1.0))
    degree = Int32(get(hp, "degree", 3))
    coef0  = Float64(get(hp, "coef0", 0.0))

    # Load MLJ SVM model
    SVM_model = @load SVC pkg=LIBSVM verbosity=0
    model = SVM_model(
        kernel = kernel_enum,
        cost   = cost,
        gamma  = gamma,
        degree = degree,
        coef0  = coef0
    )

    # --- Train machine ---
    mach = machine(model, train_df, train_y_cat)
    MLJ.fit!(mach)

    # --- Predict ---
    ŷ = MLJ.predict(mach, test_df)
    y_pred = CategoricalArray(ŷ)

    # --- Metrics ---
    # --- Compute metrics using your function ---
    accuracy, error_rate, recall, specificity, ppv, npv, f1, cm =
        confusionMatrix(y_pred, test_y_cat)

    println("Accuracy = ", round(accuracy, digits=4))

    push!(svm_results, (
        model = :SVC,
        hyperparams = hp,
        metrics = (accuracy, error_rate, recall, specificity, ppv, npv, f1, cm)
    ))


end

results = Dict(:SVC => svm_results)
println("\n===== Grid search finished =====")

for entry in results[:SVC]
    printResult(entry.model, entry.hyperparams, entry.metrics,label_names)
    #printExperimentResult(:SVC, Dict("C"=>0.1, "kernel"=>"linear"), results, class_labels=labels)
end






=== SVM experiment: kernel=linear C=0.1 ===
Accuracy = 0.926

=== SVM experiment: kernel=linear C=1.0 ===
Accuracy = 0.937

=== SVM experiment: kernel=linear C=10.0 ===
Accuracy = 0.94

=== SVM experiment: kernel=rbf C=1.0 ===
Accuracy = 0.946

=== SVM experiment: kernel=rbf C=10.0 ===
Accuracy = 0.948

=== SVM experiment: kernel=sigmoid C=1.0 ===
Accuracy = 0.552

=== SVM experiment: kernel=poly C=1.0 ===
Accuracy = 0.943

=== SVM experiment: kernel=poly C=5.0 ===
Accuracy = 0.932

===== Grid search finished =====

 Model: SVC
 Hyperparameters: Dict{String, Any}("C" => 0.1, "kernel" => "linear")
 Accuracy                : 0.926
 Error Rate              : 0.074
 Sensitivity/Recall      : 0.926
 Specificity             : 0.9738
 PPV                     : 0.9255
 NPV                     : 0.983
 F1 Score                : 0.9242

Confusion Matrix:
[275 0 8 5; 0 65 0 28; 0 0 408 0; 26 6 1 178]

┌───────────┬──────┬───────────┬──────────┬──────┐
│           │ Good │ Hazardous │ Moderate │ 

# DT

In [15]:
############# 3. DECISION TREES (6 depths) #############
default_dt = Dict(
    "rng" => Random.MersenneTwister(1)
)

dt_search_space = [
    Dict("max_depth"=>2),
    Dict("max_depth"=>3),
    Dict("max_depth"=>4),
    Dict("max_depth"=>5),
    Dict("max_depth"=>6),
    Dict("max_depth"=>8)
]

# --- Container for results ---
dt_results = []

# --- Wrap PCA / raw data as DataFrames ---
# Assume train_df and test_df are already prepared DataFrames of features

# ----------------------------
# DECISION TREE GRID SEARCH
# ----------------------------
for hp in dt_search_space
    println("\n=== Decision Tree experiment: max_depth=$(hp["max_depth"]) ===")
    
    # Merge default hyperparameters with current hp
    full_hp = merge(default_dt, hp)
    
    # Load DecisionTreeClassifier
    DT_model = @load DecisionTreeClassifier pkg=DecisionTree verbosity=0
    model = DT_model(
        max_depth = full_hp["max_depth"],
        rng       = full_hp["rng"]
    )
    
    # --- Train machine ---
    mach = machine(model, train_df, train_y_cat)
    MLJ.fit!(mach)
    
    # --- Predict ---
    ŷ = MLJ.predict(mach, test_df)
    y_pred = CategoricalArray(mode.(ŷ))  # convert MLJ probabilistic predictions to class labels

    
    # --- Compute metrics ---
    accuracy, error_rate, recall, specificity, ppv, npv, f1, cm =
        confusionMatrix(y_pred, test_y_cat)
    
    println("Accuracy = ", round(accuracy, digits=4))
    
    push!(dt_results, (
        model = :DecisionTree,
        hyperparams = hp,
        metrics = (accuracy, error_rate, recall, specificity, ppv, npv, f1, cm)
    ))
end

# Store results in dict
results[:DecisionTree] = dt_results
println("\n===== Grid search finished =====")

# ----------------------------
# Print all results
# ----------------------------
for entry in results[:DecisionTree]
    printResult(entry.model, entry.hyperparams, entry.metrics, label_names)
end




=== Decision Tree experiment: max_depth=2 ===
Accuracy = 0.828

=== Decision Tree experiment: max_depth=3 ===
Accuracy = 0.89

=== Decision Tree experiment: max_depth=4 ===
Accuracy = 0.905

=== Decision Tree experiment: max_depth=5 ===
Accuracy = 0.918

=== Decision Tree experiment: max_depth=6 ===
Accuracy = 0.92

=== Decision Tree experiment: max_depth=8 ===
Accuracy = 0.932

===== Grid search finished =====

 Model: DecisionTree
 Hyperparameters: Dict("max_depth" => 2)
 Accuracy                : 0.828
 Error Rate              : 0.172
 Sensitivity/Recall      : 0.828
 Specificity             : 0.9443
 PPV                     : 0.764
 NPV                     : 0.9638
 F1 Score                : 0.7927

Confusion Matrix:
[275 0 1 12; 2 0 0 91; 8 0 400 0; 58 0 0 153]

┌───────────┬──────┬───────────┬──────────┬──────┐
│           │ Good │ Hazardous │ Moderate │ Poor │
├───────────┼──────┼───────────┼──────────┼──────┤
│      Good │  275 │         0 │        1 │   12 │
│ Hazardous │    

# KNN

In [16]:
############# 4. kNN (6 values) #############
knn_search_space = [
    Dict("K"=>1),
    Dict("K"=>3),
    Dict("K"=>5),
    Dict("K"=>7),
    Dict("K"=>9),
    Dict("K"=>11)
]

# --- Convert targets ---
train_y_cat = categorical(vec(trainingTargets))
test_y_cat  = categorical(vec(testTargets))

# --- Container for results ---
knn_results = []

# --- kNN hyperparameter search space ---
knn_search_space = [
    Dict("K"=>1),
    Dict("K"=>3),
    Dict("K"=>5),
    Dict("K"=>7),
    Dict("K"=>9),
    Dict("K"=>11)
]

# --- Loop over K values ---
KNN_model_type = @load KNNClassifier pkg=NearestNeighborModels verbosity=0

for hp in knn_search_space
    println("\n=== kNN experiment: K=$(hp["K"]) ===")
    
    K = Int(hp["K"])
    
    # Load MLJ KNN model
    model = KNN_model_type(K=K)
    
    # --- Train machine ---
    mach = machine(model, train_df, train_y_cat)
    MLJ.fit!(mach)
    
    # --- Predict ---
    ŷ = MLJ.predict(mach, test_df)              # probabilistic predictions
    y_pred = CategoricalArray(mode.(ŷ))         # convert to class labels
    
    # --- Compute metrics ---
    accuracy, error_rate, recall, specificity, ppv, npv, f1, cm =
        confusionMatrix(y_pred, test_y_cat)
    
    println("Accuracy = ", round(accuracy, digits=4))
    
    # --- Save results ---
    push!(knn_results, (
        model = :kNN,
        hyperparams = hp,
        metrics = (accuracy, error_rate, recall, specificity, ppv, npv, f1, cm)
    ))
end

# --- Store all results ---
results[:kNN] = knn_results

# --- Print results ---
for entry in results[:kNN]
    printResult(entry.model, entry.hyperparams, entry.metrics, label_names)
end



=== kNN experiment: K=1 ===
Accuracy = 0.913

=== kNN experiment: K=3 ===
Accuracy = 0.925

=== kNN experiment: K=5 ===
Accuracy = 0.934

=== kNN experiment: K=7 ===
Accuracy = 0.93

=== kNN experiment: K=9 ===
Accuracy = 0.928

=== kNN experiment: K=11 ===
Accuracy = 0.93

 Model: kNN
 Hyperparameters: Dict("K" => 1)
 Accuracy                : 0.913
 Error Rate              : 0.087
 Sensitivity/Recall      : 0.913
 Specificity             : 0.9762
 PPV                     : 0.9116
 NPV                     : 0.9786
 F1 Score                : 0.9121

Confusion Matrix:
[273 1 1 13; 0 67 0 26; 1 0 407 0; 25 20 0 166]

┌───────────┬──────┬───────────┬──────────┬──────┐
│           │ Good │ Hazardous │ Moderate │ Poor │
├───────────┼──────┼───────────┼──────────┼──────┤
│      Good │  273 │         1 │        1 │   13 │
│ Hazardous │    0 │        67 │        0 │   26 │
│  Moderate │    1 │         0 │      407 │    0 │
│      Poor │   25 │        20 │        0 │  166 │
└───────────┴──────

In [17]:
############# 4. kNN (6 values) #############
knn_search_space = [
    Dict("K"=>1),
    Dict("K"=>3),
    Dict("K"=>5),
    Dict("K"=>7),
    Dict("K"=>9),
    Dict("K"=>11)
]

# --- Convert targets ---
train_y_cat = categorical(vec(trainingTargets))
test_y_cat  = categorical(vec(testTargets))

# --- Container for results ---
knn_results = []

# --- kNN hyperparameter search space ---
knn_search_space = [
    Dict("K"=>1),
    Dict("K"=>3),
    Dict("K"=>5),
    Dict("K"=>7),
    Dict("K"=>9),
    Dict("K"=>11)
]

# --- Loop over K values ---
KNN_model_type = @load KNNClassifier pkg=NearestNeighborModels verbosity=0

for hp in knn_search_space
    println("\n=== kNN experiment: K=$(hp["K"]) ===")
    
    K = Int(hp["K"])
    
    # Load MLJ KNN model
    model = KNN_model_type(K=K)
    
    # --- Train machine ---
    mach = machine(model, train_df, train_y_cat)
    MLJ.fit!(mach)
    
    # --- Predict ---
    ŷ = MLJ.predict(mach, test_df)              # probabilistic predictions
    y_pred = CategoricalArray(mode.(ŷ))         # convert to class labels
    
    # --- Compute metrics ---
    accuracy, error_rate, recall, specificity, ppv, npv, f1, cm =
        confusionMatrix(y_pred, test_y_cat)
    
    println("Accuracy = ", round(accuracy, digits=4))
    
    # --- Save results ---
    push!(knn_results, (
        model = :kNN,
        hyperparams = hp,
        metrics = (accuracy, error_rate, recall, specificity, ppv, npv, f1, cm)
    ))
end

# --- Store all results ---
results[:kNN] = knn_results

# --- Print results ---
for entry in results[:kNN]
    printResult(entry.model, entry.hyperparams, entry.metrics, label_names)
end



=== kNN experiment: K=1 ===
Accuracy = 0.913

=== kNN experiment: K=3 ===
Accuracy = 0.925

=== kNN experiment: K=5 ===
Accuracy = 0.934

=== kNN experiment: K=7 ===
Accuracy = 0.93

=== kNN experiment: K=9 ===
Accuracy = 0.928

=== kNN experiment: K=11 ===
Accuracy = 0.93

 Model: kNN
 Hyperparameters: Dict("K" => 1)
 Accuracy                : 0.913
 Error Rate              : 0.087
 Sensitivity/Recall      : 0.913
 Specificity             : 0.9762
 PPV                     : 0.9116
 NPV                     : 0.9786
 F1 Score                : 0.9121

Confusion Matrix:
[273 1 1 13; 0 67 0 26; 1 0 407 0; 25 20 0 166]

┌───────────┬──────┬───────────┬──────────┬──────┐
│           │ Good │ Hazardous │ Moderate │ Poor │
├───────────┼──────┼───────────┼──────────┼──────┤
│      Good │  273 │         1 │        1 │   13 │
│ Hazardous │    0 │        67 │        0 │   26 │
│  Moderate │    1 │         0 │      407 │    0 │
│      Poor │   25 │        20 │        0 │  166 │
└───────────┴──────